# Convolution Network

Convolution Neural Networks (CNN) have been very successful especially when modeling images. In this notebook we introduce CNNs and use Keras to learn the CIFAR10 data set

### Packages

In [1]:
import tensorflow as tf 
import numpy as np 
import matplotlib.pyplot as plt
from tensorflow.keras import models,layers
from tensorflow.keras.utils import Sequence
#from tensorflow.python.keras.utils import data_utils
import math
import os


INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2


In [2]:
train_dir="C:/Users/hikma/Downloads/cifar10-pngs-in-folders/cifar10/cifar10/train"
test_dir="C:/Users/hikma/Downloads/cifar10-pngs-in-folders/cifar10/cifar10/test"

cats=os.listdir(train_dir)
print(cats)
train_figs=np.array([ list(map((train_dir+"/"+cats[i]+"/").__add__,os.listdir(train_dir+"/"+cats[i]))) for i in range(0,10)])
test_figs=np.array([ list(map((test_dir+"/"+cats[i]+"/").__add__,os.listdir(test_dir+"/"+cats[i]))) for i in range(0,10)])

['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']


In [3]:
test_figs=np.transpose(test_figs)
test_figs=np.hstack(test_figs)
train_figs=np.transpose(train_figs)
train_figs=np.hstack(train_figs)
print(train_figs.shape)
train_labels=np.array([ i%10 for i in range(0,len(train_figs))])
test_labels=np.array([ i%10 for i in range(0,len(test_figs))])

(50000,)


In [4]:
#class CIFAR10Sequence(data_utils.Sequence):
class CIFAR10Sequence(Sequence):
    def __init__(self, x_set, y_set, batch_size):
        self.x, self.y = x_set, y_set
        self.batch_size = batch_size

    def __len__(self):
        return math.ceil(len(self.x) / self.batch_size)

    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) *
        self.batch_size]
        batch_y = self.y[idx * self.batch_size:(idx + 1) *
        self.batch_size]

        return np.array([plt.imread(filename) for filename in batch_x]),batch_y

In [10]:
batch_size=128
train_dataset=CIFAR10Sequence(train_figs,train_labels,batch_size)
test_dataset=CIFAR10Sequence(test_figs,test_labels,batch_size)
(f,l)=train_dataset.__getitem__(0)
f.shape

(128, 32, 32, 3)

In [6]:
def createModel():
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))

    model.add(layers.Conv2D(64, (3, 3),  activation='relu'))
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))

    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))

    model.add(layers.Flatten())

    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(10,activation='softmax'))
    
    return model

In [7]:
model=createModel()
#tf.keras.utils.plot_model(model,show_shapes=True)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 30, 30, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 13, 13, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 6, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 4, 4, 64)          36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 2, 2, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 256)               0

## Optimization

Keras can use many optimization method. In this notebook we use the __Adam__ method which can be described loosely as __adaptive__ gradient descent.

Also since the labels are __NOT__ in one_hot_encoding we use the "Sparse" version of the crossentropy loss: __SparseCategoricalCrossentropy__. Finally, if we don't specify from_logits=False then the loss function would compute softwmax before computing the loss. Since we are computing softwmax in our model already we turn this step off by specifying from_logits=False

In [8]:
# if we don't use softmax in the last layer, i.e. if the output of the
# model is NOT probabilities then use from_logits=True
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])

#history = model.fit(img_train,label_train, batch_size=128,epochs=10, 
#                    validation_data=(img_test, label_test))
history=model.fit(train_dataset,batch_size=batch_size,epochs=10)


Epoch 1/10
391/391 [==============================] - 40s 89ms/step - loss: 1.8500 - accuracy: 0.3227
Epoch 2/10
391/391 [==============================] - 34s 88ms/step - loss: 1.3322 - accuracy: 0.5261
Epoch 3/10
391/391 [==============================] - 34s 88ms/step - loss: 1.1979 - accuracy: 0.5812
Epoch 4/10
391/391 [==============================] - 34s 88ms/step - loss: 1.0922 - accuracy: 0.6199
Epoch 5/10
391/391 [==============================] - 33s 85ms/step - loss: 1.0323 - accuracy: 0.6426
Epoch 6/10
391/391 [==============================] - 57s 147ms/step - loss: 0.9784 - accuracy: 0.6629
Epoch 7/10
391/391 [==============================] - 61s 155ms/step - loss: 0.9169 - accuracy: 0.6847
Epoch 8/10
391/391 [==============================] - 57s 145ms/step - loss: 0.8764 - accuracy: 0.6958
Epoch 9/10
391/391 [==============================] - 35s 89ms/step - loss: 0.8401 - accuracy: 0.7113
Epoch 10/10
391/391 [==============================] - 34s 88ms/step - loss: 0.

### Testing the Accuracy

In [11]:
_,test_accuracy=model.evaluate(test_dataset)

79/79 [==============================] - 174s 2s/step - loss: 0.8974 - accuracy: 0.6936
